In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import os
import time
import json
import pandas as pd
from tqdm import tqdm

In [8]:
root = "drive/MyDrive/data/bank/"
url_bamosz = "https://www.bamosz.hu/alapoldal?isin="

In [10]:
with open(root + "data.json","r",encoding="utf8") as jf:
    stocks = json.load(jf)

In [11]:
for key in tqdm(stocks.keys()):
  file_with_paht = root + "/cleaned/" + key + ".csv"
  if os.path.exists(file_with_paht):
    stocks[key] = {
        "data" : pd.read_csv(file_with_paht, sep=";", decimal=",",encoding='utf-8-sig'),
        "name" : stocks[key]
    }
  else:
    stocks[key] = { "name" : stocks[key] }

100%|██████████| 93/93 [00:00<00:00, 101.41it/s]


In [22]:
stocks["HU0000720271"]["data"].dtypes

Dátum                                  object
Árfolyam                              float64
NEÉ                                     int64
Kifizetett hozam                      float64
Napi befjegy. forgalom                float64
Napi befjegy. forgalom változás(%)    float64
3 hónapos hozam                       float64
6 hónapos hozam                       float64
1 éves hozam                          float64
3 éves hozam                          float64
5 éves hozam                          float64
10 éves hozam                         float64
dtype: object

In [32]:
def reformat_csv(data):
  tmp = data.copy()
  tmp = tmp.fillna(0.0)
  tmp["Dátum"] = tmp["Dátum"].apply(lambda x: x[:4] + "-" + x[4:6] + "-" + x[6:])
  tmp["Árfolyam"] = tmp["Árfolyam"].apply(lambda x: float(str(x)))
  tmp["Nettó eszközérték"] = tmp["Nettó eszközérték"].apply(lambda x: int(str(x).replace(" ","")))
  tmp["Napi bef. jegy forgalom"] = tmp["Napi bef. jegy forgalom"].apply(lambda x: float(x))
  tmp["Napi befjegy forgalom vált."] = tmp["Napi befjegy forgalom vált."].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["3 havi"] = tmp["3 havi"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["6 havi"] = tmp["6 havi"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["1 éves"] = tmp["1 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["3 éves"] = tmp["3 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["5 éves"] = tmp["5 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  tmp["10 éves"] = tmp["10 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
  return tmp

In [35]:
pbar = tqdm(stocks.keys())
for key in pbar:
  pbar.set_description(key)
  if 'data' in stocks[key].keys():
    stocks[key]["new_data"] = pd.read_html(url_bamosz+key, 
                                           attrs={"class" : "dataTable2"}, 
                                           decimal=",", 
                                           thousands='.')[1]
  time.sleep(1)

HU0000718523:  11%|█         | 10/93 [00:30<04:12,  3.04s/it]


URLError: ignored

In [13]:
url_bamosz+list(stocks.keys())[0]

'https://www.bamosz.hu/alapoldal?isin=HU0000720271'

In [16]:
url_bamosz+"HU0000720271"

'https://www.bamosz.hu/alapoldal?isin=HU0000720271'

In [15]:
tmp = pd.read_html(url_bamosz+"HU0000720271", attrs={"class" : "dataTable2"}, decimal=",", thousands='.')[1]

In [17]:
tmp.columns = tmp.values[0]
tmp = tmp[1:].copy().iloc[::-1]
tmp

,Dátum,Árfolyam,Nettó eszközérték,Kifizetett hozam,Napi bef. jegy forgalom,Napi befjegy forgalom vált.,3 havi,6 havi,1 éves,3 éves,5 éves,10 éves
253,20220106,5.058413,18 731 240 924,NaN,1949201222,"11,6%","-6,3%","-2,7%","-2,5%","8,8%",NaN,NaN
252,20220107,5.087926,16 869 093 719,NaN,-1971433480,"-10,5%","-5,4%","-2,4%","-2,6%","8,9%",NaN,NaN
251,20220110,5.100149,16 879 394 822,NaN,-30224434,"-0,2%","-5,3%","-2,1%",-2%,"8,8%",NaN,NaN
250,20220111,5.08728,16 836 805 493,NaN,1766,0%,"-5,7%","-2,3%","-2,4%","8,6%",NaN,NaN
249,20220112,5.120966,16 948 292 463,NaN,158,0%,"-4,9%","-1,6%","-1,5%","8,9%",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5,20221229,6.555339,18 263 225 679,NaN,-1082,0%,"12,8%","15,7%","31,7%","14,8%",NaN,NaN
4,20221230,6.548128,18 234 996 351,NaN,-8139426,0%,"11,7%","15,9%","31,9%","14,2%",NaN,NaN
3,20230102,6.560608,18 269 752 205,NaN,1989,0%,"11,9%","16,1%","31,3%","14,4%",NaN,NaN
2,20230103,6.599477,18 377 992 768,NaN,-472,0%,"12,6%","16,8%","31,3%","14,9%",NaN,NaN


In [18]:
data = tmp.copy()

In [30]:
tmp = data.copy()
tmp = tmp.fillna(0.0)
tmp["Dátum"] = tmp["Dátum"].apply(lambda x: x[:4] + "-" + x[4:6] + "-" + x[6:])
tmp["Árfolyam"] = tmp["Árfolyam"].apply(lambda x: float(str(x)))
tmp["Nettó eszközérték"] = tmp["Nettó eszközérték"].apply(lambda x: int(str(x).replace(" ","")))
tmp["Napi bef. jegy forgalom"] = tmp["Napi bef. jegy forgalom"].apply(lambda x: float(x))
tmp["Napi befjegy forgalom vált."] = tmp["Napi befjegy forgalom vált."].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
tmp["3 havi"] = tmp["3 havi"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
tmp["6 havi"] = tmp["6 havi"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
tmp["1 éves"] = tmp["1 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
tmp["3 éves"] = tmp["3 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
tmp["5 éves"] = tmp["5 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
tmp["10 éves"] = tmp["10 éves"].apply(lambda x: float(str(x).replace("%","").replace(",", ".")))
tmp

,Dátum,Árfolyam,Nettó eszközérték,Kifizetett hozam,Napi bef. jegy forgalom,Napi befjegy forgalom vált.,3 havi,6 havi,1 éves,3 éves,5 éves,10 éves
253,2022-01-06,5.058413,18731240924,0.0,1.949201e+09,11.6,-6.3,-2.7,-2.5,8.8,0.0,0.0
252,2022-01-07,5.087926,16869093719,0.0,-1.971433e+09,-10.5,-5.4,-2.4,-2.6,8.9,0.0,0.0
251,2022-01-10,5.100149,16879394822,0.0,-3.022443e+07,-0.2,-5.3,-2.1,-2.0,8.8,0.0,0.0
250,2022-01-11,5.087280,16836805493,0.0,1.766000e+03,0.0,-5.7,-2.3,-2.4,8.6,0.0,0.0
249,2022-01-12,5.120966,16948292463,0.0,1.580000e+02,0.0,-4.9,-1.6,-1.5,8.9,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5,2022-12-29,6.555339,18263225679,0.0,-1.082000e+03,0.0,12.8,15.7,31.7,14.8,0.0,0.0
4,2022-12-30,6.548128,18234996351,0.0,-8.139426e+06,0.0,11.7,15.9,31.9,14.2,0.0,0.0
3,2023-01-02,6.560608,18269752205,0.0,1.989000e+03,0.0,11.9,16.1,31.3,14.4,0.0,0.0
2,2023-01-03,6.599477,18377992768,0.0,-4.720000e+02,0.0,12.6,16.8,31.3,14.9,0.0,0.0


In [31]:
tmp.dtypes

Dátum                           object
Árfolyam                       float64
Nettó eszközérték                int64
Kifizetett hozam               float64
Napi bef. jegy forgalom        float64
Napi befjegy forgalom vált.    float64
3 havi                         float64
6 havi                         float64
1 éves                         float64
3 éves                         float64
5 éves                         float64
10 éves                        float64
dtype: object